In [ ]:
! pip install prance mistral_common openapi-spec-validator pyaudio
! pip install --no-build-isolation --force-reinstall \
    "boto3>=1.28.57" \
    "awscli>=1.29.57" \
    "botocore>=1.31.57"

In [ ]:

import prance
from typing import List
from mistral_common.tokens.tokenizers.mistral import MistralTokenizer
from mistral_common.protocol.instruct.request import ChatCompletionRequest
from mistral_common.protocol.instruct.tool_calls import Function, Tool
from mistral_common.protocol.instruct.messages import UserMessage
import json
import requests
import functools
import boto3
import json
import pyaudio
import wave
import os
#bedrock = boto3.client(service_name="bedrock-runtime", region_name='us-east-1')
bedrock = boto3.client(service_name="bedrock-runtime")


In [ ]:
def get_user_messages(queries: List[str]) -> List[UserMessage]:
    user_messages=[]
    for query in queries:
        user_message = UserMessage(content=query)
        user_messages.append(user_message)
    return user_messages

In [ ]:
def generate_tools(objs, function_end_point)-> List[Tool]:
    params = ['operationId', 'description',  'parameters']

    parser = prance.ResolvingParser(function_end_point, backend='openapi-spec-validator')
    spec = parser.specification
    user_tools = []
    for obj in objs:
        resource, field = obj
        path = '/' + resource + '/{' + field + '}'
        function_name=spec['paths'][path]['get'][params[0]]
        function_description=spec['paths'][path]['get'][params[1]]
        function_parameters=spec['paths'][path]['get'][params[2]]
        func_parameters = {
        "type": "object",
        "properties": {
            function_parameters[0]['name']: {
                "type": function_parameters[0]['schema']['type'],
                "description": function_parameters[0]['description']
            }
        },
        "required": [function_parameters[0]['name']]
    }
        user_function= Function(name = function_name, description = function_description, parameters = func_parameters, )
        user_tool = Tool(function = user_function)
        user_tools.append(user_tool)
    return user_tools


In [ ]:
def bedrock_apicall(prompt):
    body = json.dumps({
        "prompt": prompt,
        "max_tokens": 512,
        "top_p": 0.8,
        "temperature": 0.5,
    })
    
    # modelId = "mistral.mistral-large-instruct-v0:2"
    modelId = "mistral.mistral-large-2402-v1:0"
    
    accept = "application/json"
    contentType = "application/json"
    
    response = bedrock.invoke_model(
        body=body,
        modelId=modelId,
        accept=accept,
        contentType=contentType
    )
    
    #print(json.loads(response.get('body').read()))
    return response.get('body').read()
    

In [ ]:
def generate_tool_prompt(queries):
    return_objs = [['pet','petId'], ['user', 'username'], ['store/order','orderId']]
    function_end_point = 'https://petstore3.swagger.io/api/v3/openapi.json'

    user_messages=get_user_messages(queries)
    user_tools = generate_tools(return_objs, function_end_point)

    tokenizer = MistralTokenizer.v3()
    completion_request = ChatCompletionRequest(tools=user_tools, messages=user_messages,)
    tokenized = tokenizer.encode_chat_completion(completion_request)
    _, text = tokenized.tokens, tokenized.text
    prompt=text
    return prompt

In [ ]:
def voice_capture():


    # Set audio parameters
    CHUNK = 1024
    FORMAT = pyaudio.paInt16
    CHANNELS = 1
    RATE = 16000
    RECORD_SECONDS = 7

    # Initialize PyAudio
    p = pyaudio.PyAudio()

    # Open audio stream
    stream = p.open(format=FORMAT,
                    channels=CHANNELS,
                    rate=RATE,
                    input=True,
                    frames_per_buffer=CHUNK)

    print("Recording...")

    # Create a list to store audio frames
    frames = []

    # Record audio for the specified duration
    for i in range(0, int(RATE / CHUNK * RECORD_SECONDS)):
        data = stream.read(CHUNK)
        frames.append(data)

    print("Finished recording.")

    # Stop and close the audio stream
    stream.stop_stream()
    stream.close()
    p.terminate()

    # Save the recorded audio to a WAV file
    wf = wave.open("output.wav", 'wb')
    wf.setnchannels(CHANNELS)
    wf.setsampwidth(p.get_sample_size(FORMAT))
    wf.setframerate(RATE)
    wf.writeframes(b''.join(frames))
    wf.close()


In [ ]:
def getPetById(petId: int) -> str:
    try:
        method = 'GET'
        headers=None
        data=None
        url =  'https://petstore3.swagger.io/api/v3/pet/' + str(petId)
        response = requests.request(method, url, headers=headers, data=data)
        # Raise an exception if the response was unsuccessful
        response.raise_for_status()
        #response = make_api_call('GET', url + str(petId))
        if response.ok :
            json_response = response.json()
            if petId == json_response['id']:
                return json_response
        return json.dumps({'error': 'Pet id not found.'})
    except requests.exceptions.HTTPError as e:
        if response.status_code == 404:
            return json.dumps({'error': 'Pet id not found.'})
        else:
            return json.dumps({'error': 'Error with API.'})
def getUserByName(username: str) -> str:
    try:
        url = 'https://petstore3.swagger.io/api/v3/user/' + username
        response = requests.get(url)
        # Raise an exception if the response was unsuccessful
        response.raise_for_status()
        if response.ok :
            json_response = response.json()
            if username == json_response['username']:
                return json_response
        return json.dumps({'error': 'Username id not found.'})
    except requests.exceptions.HTTPError as e:
        if response.status_code == 404:
            return json.dumps({'error': 'Username not found.'})
        else:
            return json.dumps({'error': 'Error with API.'})

names_to_functions = {
  'getPetById': functools.partial(getPetById, petId=''),
  'getUserByName': functools.partial(getUserByName, username='')  
}

In [ ]:
def process_results(result):
    result_format = result['outputs'][0]['text']

    result_tool_calls = (result_format.replace("[TOOL_CALLS] ",""))

    result_tool_calls = eval(result_tool_calls)

    function_name = result_tool_calls[0]["name"]
    function_params = (result_tool_calls[0]["arguments"]) 
    function_result = names_to_functions[function_name](**function_params)
    print(function_result)


In [ ]:
def voice_query():
    url = "http://localhost:5000/whisper"
    #files = {'file': open('/path/to/filename.mp3', 'rb')}
    #files = {'file': open('../data/test1.flac', 'rb')}
    files = {'file': open('output.wav', 'rb')}
    response = requests.post(url, files=files)

    response_query = ""
    if response.status_code == 200:
        query = json.loads(response.text)
        for result in query['results']:
            transcript = result['transcript']
            response_query = response_query + transcript
        return response_query    
    else:
        print(f"Error: {response.status_code} - {response.text}")

In [ ]:
    voice_capture()
    voice_response = voice_query()
    print("speech recognition output- " + voice_response)
    #queries = ["What's the status of my Pet 1?", "Find information of user user1?" ,  "What's the status of my Store Order 3?"]
    queries = []
    queries.append(voice_response)
    return_objs = [['pet','petId'], ['user', 'username'], ['store/order','orderId']]

    prompt_text = generate_tool_prompt(queries)
    response = bedrock_apicall(prompt_text)
    response_json = json.loads(response)
    #print(response_json)
    process_results(response_json)
    #execute_generator(queries, return_objs)
    